# TASEPy applications

In [ ]:
As an application of TASEPy package, we show how to infer the initiation ra

In [59]:
# import methods from TASEPy

from TASEPy import psa_compute
from TASEPy import local_density
from TASEPy import mean_density
from TASEPy import current

# to measure computation time
import time 

# root finding algorithm
from scipy import optimize # to 

First, we import a file containing hopping rates for the YAL008W gene of *S. cerevisiae*, which have been estimated in Ciandrini L, Stansfield I, Romano MC (2013) Ribosome Traffic on mRNAs Maps to Gene Ontology: Genome-wide Quantification of Translation Initiation Rates and Polysome Size Regulation. PLoS Comput Biol 9(1): e1002866. https://doi.org/10.1371/journal.pcbi.1002866

In [46]:
# imports hopping rates

file = open('applications/YAL008W_rates.dat','r')

rates = []
for line in file:
    linelist = line.split()
    codon = linelist[0]
    rate = float(linelist[1])
    rates.append(rate)
file.close()

Next, we set particle size $\ell=9$ as in the reference above, and choose the order of PSA $K=3$. This takes about 11 seconds to solve on a laptop.

In [95]:
# computes the PSA for order K = 3

L = len(rates) # lattice size
ll = 9 # particle size
K = 3 # maximum PSA order

print('Lattice size:',L,'codons')

start = time.time()
rhocoeff, Jcoeff = psa_compute(rates, K, ll)
end = time.time()
print('Computation time:',round(end-start,3),'seconds')

Lattice size: 198 codons
Computation time: 11.16 seconds


In [96]:
# computes mean density coefficients

mean_rhocoeff = []
for order in range(0,K+1):
    rhocoeff_sum = 0
    for site in range(L):
        rhocoeff_sum += rhocoeff[site][order]
    mean_rhocoeff.append(rhocoeff_sum/L)
print(mean_rhocoeff)

[0.0, 0.1806069864721105, -0.20947619500711065, 0.32647921092495436]


Next, we set up the function whose root we want to find: $f(x)=\rho(x)-\rho_{\text{exp}}$, where $\rho(x)$ is the mean density computed using TASEPy for a given value of the initiation rate $x$, and $\rho_{\text{exp}}$ is the mean density measured experimentally, see the cited Ciandrini L et (2013) reference for more details. Since $f(x)$ is a polynomial, we can also find it derivative, in which case we can use the Newton-Raphson method for solving $f(x)=0$.

In [87]:
# experimental mean density
rhoexp = 0.023226

def f(x,a):
    result = -rhoexp
    for order,coeff in enumerate(a):
        result += coeff * x**order
    return result

def fprime(x,a):
    result = 0
    for order,coeff in enumerate(a):
        if order > 0:
            result += coeff * order * x**(order-1)
    return result

As the initial guess, we can use the first-order approximation of the PSA.

In [98]:
# initial guess
x0 = rhoexp/mean_rhocoeff[1]
print('Initial value:',x0)

Initial value: 0.12859967631200458


Finally, we call the optimize function from the scipy package to find the root.

In [92]:
alpha = optimize.newton(f, x0, fprime, args=(mean_rhocoeff,))
print('Optimal value:',alpha)

Optimal value: 0.14818676243771686


This value is very close to the one obtained from stochastic simulations in the cited Ciandrini et al (2013) paper. The relative error is about 1.5\%.

In [93]:
# value reported in Ciandrini L, Stansfield I, Romano MC (2013) 
# Ribosome Traffic on mRNAs Maps to Gene Ontology: Genome-wide 
# Quantification of Translation Initiation Rates and Polysome 
# Size Regulation. PLoS Comput Biol 9(1): e1002866. 
# https://doi.org/10.1371/journal.pcbi.1002866

alpha_paper = 0.150499
per_err = 100 * abs(alpha_paper-alpha)/alpha_paper
print('Relative percentage error: ',per_err)

Relative percentage error:  1.5363806817873418
